<a href="https://colab.research.google.com/github/MarinaShtei/Team1CloudPoj/blob/master/fun_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install firebase
!pip install jinja2
!rm -r '/content/gameAppUpdated' 2> /dev/null #remove folder on restart
!git clone --branch main https://github.com/AmitPerets/gameAppUpdated.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'gameAppUpdated'...
remote: Enumerating objects: 350, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 350 (delta 134), reused 133 (delta 49), pack-reused 28
Receiving objects: 100% (350/350), 89.04 KiB | 1.82 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [ ]:
from firebase import firebase
import google.colab.output as output
from IPython.display import JSON

class DB_Connector():
  connection = firebase.FirebaseApplication('https://puzzleton-45589-default-rtdb.firebaseio.com/',None)
  user = None
  name = None
  cuurent_index = -1

  @classmethod
  def login(cls,username,password):
    try:
      cls.user = cls.connection.get(f"/users/{username}",None)
      if cls.user["password"] == password:
        cls.name = username
        return True
    except:
      return False

  @classmethod
  def getUserName(cls):
    return cls.name

  @classmethod
  def isAdmin(cls):
    return cls.user["admin"]

  @classmethod
  def logout(cls):
     cls.user = None

  @classmethod
  def addQuest(cls,data):
    try:
      cls.connection.put("/questions",str(len(cls.getAllQuestions())),data)
      return True
    except:
      return False

  @classmethod
  def getAllTitles(cls):
    questions = cls.connection.get("/questions",None)
    return JSON([question["title"] for question in questions])
  
  @classmethod
  def update_question(cls,title,options,correct):
    href = "/questions/"+str(cls.current_index)
    data = {"title":title,"correct":int(correct)}
    for i,option in enumerate(options):
      data.update({"Option"+str(i+1):option})
    cls.connection.patch(href,data)

  @classmethod
  def getQuestion(cls,title):
    questions = cls.connection.get("/questions",None)
    for i,question in enumerate(questions):
      if question["title"] == title:
        cls.current_index = i
        return JSON(question)

  @classmethod
  def getAllQuestions(cls):
    return  cls.connection.get("/questions",None)

output.register_callback('db.login',DB_Connector.login) 
output.register_callback('db.logout',DB_Connector.logout) 
output.register_callback('db.update_question',DB_Connector.update_question)
output.register_callback('db.isAdmin',DB_Connector.isAdmin)
output.register_callback('db.getUserName',DB_Connector.getUserName)
output.register_callback('db.addQuest',DB_Connector.addQuest)
output.register_callback('db.getAllTitles',DB_Connector.getAllTitles)
output.register_callback('db.getQuestion',DB_Connector.getQuestion)
output.register_callback('db.getAllQuestions',DB_Connector.getAllQuestions)

In [ ]:
import random
from IPython.display import JSON

questions = DB_Connector.getAllQuestions()
asked = set()
answers = []

def generate_question():
  if len(asked) >= len(questions):
    return None
  question = random.choice(questions)
  while question["title"] in asked:
    question = random.choice(questions)
  asked.add(question["title"])
  return JSON(question)


def submit_question(title,answer,type_):
  answers.append({'title':title,'answer':answer,'type':type_})

def graph_values():
  wrong = 0
  correct = 0
  for answer in answers:
    if answer["type"] == "wrong-answer":
      wrong+=1
    else:
      correct+=1
  return JSON({"correct":correct,"wrong":wrong})

def view_timeline():
  show_page('timeline',answers)

def clear():
  global asked
  global answers
  asked = set()
  answers = []

def progress():
    percentage = (len(asked) / len(questions)) * 100
    return percentage

output.register_callback('progress',progress)
output.register_callback('graph_values',graph_values)
output.register_callback('clear',clear)
output.register_callback('submit_question',submit_question)
output.register_callback('view_timeline',view_timeline)
output.register_callback('generate_question',generate_question)

In [ ]:
from IPython.display import HTML, Javascript, clear_output
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader('/content/gameAppUpdated/html'))

def show_page(file_name, options=None):
    clear_output()
    template = env.get_template(f'{file_name}.html') #load page
    html = HTML(template.render(options=options)) #render page
    display(html) #display after rendering page
    display(Javascript(f'/content/gameAppUpdated/js/utils.js')) #load utils.js
    display(Javascript(f'/content/gameAppUpdated/js/{file_name}.js')) #load page.js


output.register_callback("show_page", show_page)
show_page("login")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>